In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as Image
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam

In [ ]:
os.chdir('/home/workspace/user-workspace')
os.getcwd()
os.listdir()

In [ ]:
path = '/home/workspace/data/.task138/sample_train_space.csv'
df = pd.read_csv(path)

# data/.task138/sample_space/AIA2012/{image_file_name}.jpg
# data/.task138/sample_space/HMIB2012/{image_file_name}.jpg
# data/.task138/sample_train_space.csv


In [ ]:
path = '/home/workspace/data/.task138/sample_train_space.csv'
df = pd.read_csv(path)


In [ ]:
pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 100)
display(df.head(100))

In [ ]:
for i in df[df.label == True].index:
    print(i)

In [ ]:
len(df[df.label == True]), len(df[df.label == False])

In [ ]:
aia_path = []
hmib_path = []
for i in range(len(df)):
    apath = os.path.join('/home/workspace/data/.task138/sample_space/sample/AIA2012', df.AIA.iloc[i])
    hpath = os.path.join('/home/workspace/data/.task138/sample_space/sample/HMIB2012/', df.HMIB.iloc[i])
    
    aia_path.append(apath)
    hmib_path.append(hpath)
    
print(len(aia_path))
print(len(hmib_path))

In [ ]:
def load_image(img_path):
    try:
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (256, 256))
        return img

    except:
        print(f'Error: {img_path}')
        return np.zeros((256, 256, 3), dtype=float)

In [ ]:
plt.figure(figsize=(20,20))
for i in range(9):
    plt.subplot(3,3,i+1)
    img = Image.imread(aia_path[i+5310])
    print(img.shape)
    print(aia_path[i+5310])
    plt.imshow(img)

plt.show()

In [ ]:
plt.figure(figsize=(20,20))
# 원래 흑백이미지이기 때문에 shape가 (512,512)인데 이것을 (512,512,3)으로 바꿈
for i in range(9):
    plt.subplot(3,3,i+1)
    img = Image.imread(hmib_path[i+5310])
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    plt.imshow(img)

plt.show()

In [ ]:
# 태양의 크기가 안맞음 이방법은 X
plt.figure(figsize=(20,20))
for i in range(9):
    plt.subplot(3,3,i+1)
    img0 = Image.imread(aia_path[i+5310])
    img = Image.imread(hmib_path[i+5310])
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
#     plt.imshow(img-img0)
    plt.imshow(img0-img)
plt.show()

In [ ]:
df.HMIB = 'HMIB2012/' + df.HMIB
df.AIA = 'AIA2012/' + df.AIA
df

In [ ]:
for i in range(len(df)):
    if df.label.iloc[i] == False:
        df.label.iloc[i] = '0'
    else:
        df.label.iloc[i] = '1'

In [ ]:
df.info()

## Multi-input imagedatagenerator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = (224,224)
batch_size = 32
directory = '/home/workspace/data/.task138/sample_space/sample/'
train_datagen = ImageDataGenerator(rescale=1./255., validation_split=0.2)


generator_tra1 = train_datagen.flow_from_dataframe(
                                    df,
                                    directory,
                                    x_col = 'HMIB',
                                    y_col = 'label',
                                    subset = 'training',
                                    target_size = image_size,
                                    color_mode = 'rgb',
                                    class_mode = 'binary',
                                    batch_size = batch_size,
                                    shuffle = True,
                                    seed = 42)

generator_tra2 = train_datagen.flow_from_dataframe(
                                    df,
                                    directory,
                                    x_col = 'AIA',
                                    y_col = 'label',
                                    subset = 'training',
                                    target_size = image_size,
                                    color_mode = 'rgb',
                                    class_mode = 'binary',
                                    batch_size = batch_size,
                                    shuffle = True,
                                    seed = 42)

print(generator_tra1.n)
print(generator_tra2.n)

In [ ]:
generator_val1 = train_datagen.flow_from_dataframe(
                                    df,
                                    directory,
                                    x_col = 'HMIB',
                                    y_col = 'label',
                                    subset = 'validation',
                                    target_size = image_size,
                                    color_mode = 'rgb',
                                    class_mode = 'binary',
                                    batch_size = batch_size,
                                    shuffle = True,
                                    seed = 42)

generator_val2 = train_datagen.flow_from_dataframe(
                                    df,
                                    directory,
                                    x_col = 'AIA',
                                    y_col = 'label',
                                    subset = 'validation',
                                    target_size = image_size,
                                    color_mode = 'rgb',
                                    class_mode = 'binary',
                                    batch_size = batch_size,
                                    shuffle = True,
                                    seed = 42)

print(generator_val1.n)
print(generator_val2.n)

In [ ]:
def two_image_generator(gen1, gen2):

    while True:
        X1, Y = gen1.next()
        X2, Y = gen2.next()
        
        yield [X1,X2], Y  #X1i[1] is the label

train_generator = two_image_generator(generator_tra1,generator_tra2)
valid_generator = two_image_generator(generator_val1,generator_val2)

## feature_model - DenseNet201

In [ ]:
# Make model == feature model
feature_model = keras.applications.DenseNet201(weights="imagenet", include_top=False)
feature_model.trainable= False

In [ ]:
train_labels = []
train_hmib_features = []
train_aia_features = []

for i in tqdm(range(generator_tra1.n//batch_size)):
    
    [x1, x2] , y = train_generator.__next__()
    train_labels.extend(y)
#     print(x1.shape, x2.shape)  #--> (32, 224, 224, 3) (32, 224, 224, 3)
    
    feature1, feature2 = feature_model.predict(x1), feature_model.predict(x2)
#     print(feature1.shape, feature2.shape) # --> (32, 7, 7, 1920) (32, 7, 7, 1920)
    train_hmib_features.extend(feature1)
    train_aia_features.extend(feature2)
    
train_hmib_features = np.array(train_hmib_features)
train_aia_features = np.array(train_aia_features)
train_labels = np.array(train_labels)

print(train_hmib_features.shape)
print(train_aia_features.shape)
print(train_labels.shape)

In [ ]:
valid_labels = []
valid_hmib_features = []
valid_aia_features = []

for i in tqdm(range(generator_val1.n//batch_size)):
    
    [x1, x2] , y = valid_generator.__next__()
    valid_labels.extend(y)
#     print(x1.shape, x2.shape)  #--> (32, 224, 224, 3) (32, 224, 224, 3)
    
    feature1, feature2 = feature_model.predict(x1), feature_model.predict(x2)
#     print(feature1.shape, feature2.shape)  #--> (32, 7, 7, 1920) (32, 7, 7, 1920)
    valid_hmib_features.extend(feature1)
    valid_aia_features.extend(feature2)
    
valid_hmib_features = np.array(valid_hmib_features)
valid_aia_features = np.array(valid_aia_features)
valid_labels = np.array(valid_labels)

print(valid_hmib_features.shape)
print(valid_aia_features.shape)
print(valid_labels.shape)

In [ ]:
# os.mkdir('./blackspot_models_and_features')

In [ ]:
hft_path = './blackspot_models_and_features/train_features_hmib.npy'
aft_path = './blackspot_models_and_features/train_features_aia.npy'
tlabel_path = './blackspot_models_and_features/train_labels.npy'

hfv_path = './blackspot_models_and_features/valid_features_hmib.npy'
afv_path = './blackspot_models_and_features/valid_features_aia.npy'
vlabel_path = './blackspot_models_and_features/valid_labels.npy'

In [ ]:
### Save feature vectors

In [ ]:
np.save(hf_path,train_hmib_features )
np.save(af_path,train_aia_features )
np.save(label_path,train_labels )

In [ ]:
np.save(hfv_path,valid_hmib_features )
np.save(afv_path,valid_aia_features )
np.save(vlabel_path,valid_labels )

In [ ]:
### Load feature vectors

In [ ]:
train_hmib_features = np.load(hft_path)
train_aia_features = np.load(aft_path)
train_labels = np.load(tlabel_path)

print("complete train data")

valid_hmib_features = np.load(hfv_path)
valid_aia_features = np.load(afv_path)
valid_labels = np.load(vlabel_path)

print("complete valid data")

## Classifier

In [ ]:
def build_multi_input_model(feature_model, num_outputs=1):
    
    inputs1 = Input(shape=feature_model.output.shape[1:])
    inputs2 = Input(shape=feature_model.output.shape[1:])
    merged = Concatenate(axis=1)([inputs1, inputs2])
    
    x = GlobalAveragePooling2D()(merged)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    
    outputs = Dense(num_outputs, activation='sigmoid')(x)

    model = Model(inputs=[inputs1, inputs2],
                 outputs = outputs)
    
    init_lr = 4e-4
    opt = Adam(lr=init_lr)
    model.compile(optimizer=opt, 
                  loss= 'binary_crossentropy',
                  metrics='accuracy')
    
    
    return model
# feature_model.output.shape[1:]

In [ ]:
model = build_multi_input_model(feature_model)

model.summary()

In [ ]:
weight_path="./blackspot_models_and_features/best_model"

cp = ModelCheckpoint(weight_path, monitor='val_accuracy', verbose=1, 
                             save_best_only=True, save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, 
                                   patience=5, 
                                   min_lr=2e-6)
es = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=50) 
callbacks_list = [cp, es, reduceLROnPlat]

In [ ]:
initial_epochs = 200

history = model.fit(x =[train_hmib_features, train_aia_features],
                    y = train_labels,
                    validation_data = ([valid_hmib_features, valid_aia_features], valid_labels),
                    epochs = initial_epochs,
                    batch_size = batch_size,
                    callbacks = callbacks_list
                    )

model.summary()

In [ ]:
model2 = build_multi_input_model(feature_model)
# 가중치 로드

model2.load_weights(weight_path)

# 모델 평가
print(model2.evaluate([valid_hmib_features, valid_aia_features], valid_labels))
model2.summary()

In [ ]:
model2.save('./blackspot_models_and_features/model_best_blackspot_binary.h5')

In [ ]:
# 학습 곡선
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
## Fine tuning

In [ ]:
model3= load_model('./blackspot_models_and_features/model_best_blackspot_binary.h5')
print(model3.evaluate([valid_hmib_features, valid_aia_features], valid_labels))
model3.summary()

In [ ]:
def build_finetuning(model, num_outputs=2):
    
    inputs1 = Input(shape=(224, 224,3))
    inputs2 = Input(shape=(224, 224,3))
    
    x1 = feature_model(inputs1)
    x2 = feature_model(inputs2)
    outputs = model([x1,x2])

    model_fine = Model(inputs=[inputs1,inputs2],
                 outputs = outputs)
    init_lr = 1e-6
    opt = Adam(lr=init_lr)
    model_fine.compile(optimizer=opt, 
                    loss= 'binary_crossentropy',
                    metrics='accuracy')
        
    return model_fine

In [ ]:
# finetuning 모델 구축
finetune = build_finetuning(model3)

In [ ]:
feature_model.trainable = True

print("Number of layers in the base model: ", len(feature_model.layers))
fine_tune_at = 350

for layer in feature_model.layers[:fine_tune_at]:
    layer.trainable =  False

finetune.summary()

In [ ]:
weight_path="./blackspot_models_and_features/bes_model_finetuned"

cp = ModelCheckpoint(weight_path, monitor='val_accuracy', verbose=1, 
                             save_best_only=True, save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, 
                                   patience=10, 
                                   min_lr=2e-7)
es = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=50) 
callbacks_list = [cp, es, reduceLROnPlat]

In [ ]:
fine_tune_epochs = 50

total_epochs =  initial_epochs + fine_tune_epochs

history_fine = finetune.fit(train_generator,
                    validation_data = valid_generator,
                    epochs = total_epochs,
                    steps_per_epoch = generator_tra1.n//batch_size,
                    validation_steps = generator_val1.n//batch_size,
                    initial_epoch= history.epoch[-1],
                    batch_size = batch_size,
                    callbacks = callbacks_list
                    )

finetune.summary()

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')

plt.savefig('DanseNet_graph.jpg')
plt.show()

In [ ]:
model4 = build_finetuning(model3)

# 가중치 로드
model4.load_weights(weight_path)

# 모델 평가
model4.evaluate(valid_generator)

In [ ]:
model4.save('./blackspot_models_and_features/model_best_tuned.h5')